In [4]:
import os
import pandas as pd
import numpy as np
import scipy
import datetime
from sklearn.model_selection import train_test_split
os.chdir('C:/Users/dhruv/Desktop/Kaggle/TalkingData/data')

In [2]:
train = pd.read_csv('train_sample.csv')
#train, test = train_test_split(train, test_size = 0.30, random_state = 42)
test = pd.read_csv('test.csv')

In [5]:
def dataPreProcessTime(df):
    df['click_time'] = (pd.to_datetime(df['click_time']) - datetime.datetime(2017,1,1,0,0,0)).dt.seconds
    return df

train = dataPreProcessTime(train)
test = dataPreProcessTime(test)

In [6]:
from scipy.sparse import csr_matrix

X_train = train[['ip',
 'app',
 'device',
 'os',
 'channel',
 'click_time']]

X_test = test[['ip',
 'app',
 'device',
 'os',
 'channel',
 'click_time']]

y_train = train[['is_attributed']]

#y_test = test[['is_attributed']]

In [7]:
print(X_train.shape)
print(X_test.shape)

(100000, 6)
(18790469, 6)


In [8]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

C:\Users\dhruv\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [ ]:
pred = clf.predict(X_test)

In [ ]:
import pickle

with open('outfile', 'wb') as fp:
    pickle.dump(pred, fp)